<a href="https://colab.research.google.com/github/truefoxik-ctrl/Makarkina_Tatiana_Study_HSE/blob/master/_5_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%9E%D0%9E%D0%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Импортируем все необходимые библиотеки

In [ ]:
from datetime import datetime
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import re

Создадим класс Account

В конструкторе проверка на валидный формат имени (кириллица и латиница), а также начальный баланс

Дефолтное значение баланса 0, поэтому при создании экземпляра класса можно не указывать этот параметр

но на случай того, если начальный баланс ненулевой, добавлена проверка на отрицательное значение

In [ ]:
class Account:

    __instance_count = 1000 #счетчик для формирования номера

    def __init__(self, account_holder: str, balance: float = 0):
        self.account_holder = account_holder
        self.__balance = balance

        pattern = r'(^([А-Я]{1}[а-яё]{1,23}\s[А-Я]{1}[а-яё]{1,23}|[A-Z]{1}[a-z]{1,23}\s[A-Z]{1}[a-z]{1,23})$)'
        result = re.findall(pattern, account_holder)
        total_result = re.search(pattern, account_holder)
        if result == []:
            raise ValueError('Введите корректное имя')
        else:
            pass

        if balance < 0:
            raise ValueError ('Начальный баланс не может быть отрицательным!')
        else:
            self.operations_history = [] #если все проверки успешны, создался пустой список для хранения истории операций

        self.__account_count = Account.__instance_count + 1 #присваиваем номер счета
        Account.__instance_count = self.__account_count #счетчик класса увеличиваем, чтобы для следующего экемпляра взять корректное значение

    def account_number(self): #метод для вывода номера счета отдельного экземпляра класса
        self.account_number = str(f"ACC-{self.__account_count}")
        print(self.account_number)

    def get_count(): #метод для получения общего количества созданных счетов
        print(f"Количество счетов: {Account.__instance_count - 1000} ")


    def deposit(self, amount): #метод для пополнения счета
        type = 'deposit'
        amount = float(amount)
        now = datetime.now()

        if amount < 0:
            raise ValueError('Нельзя внести на счет отрицательную сумму!')
        elif amount == 0:
            print('Необходимо внести сумму')
        elif amount > 0:
            self.__balance += amount
            status = 'success'

        self.operations_history.append([f"{status}, {now}, {type}, {amount}, {self.__balance}"]) #вносим операцию об успешном пополнении в историю
        print(f"Внесено {amount} руб.\nОстаток на счете: {self.__balance} руб.")

        return self.__balance


    def withdraw(self, amount): #метод для снятия средств со счета
        type = 'withdraw'
        amount = float(amount)
        now = datetime.now()

        if amount < 0:
            raise ValueError('Нельзя снять со счета отрицательную сумму!')
        elif amount == 0:
            print('Пожалуйста, введите другую сумму для снятия')
        elif amount > 0: #для этих операций согласно заданию, будем отражать в истории как успешные, так и неуспешные операции (кроме не валидных)

            if amount <= self.__balance:
                self.__balance -= amount
                status = 'success'
                self.operations_history.append([f"{status}, {now}, {type}, {amount}, {self.__balance}"])
                print(f"Снято {amount} руб.\nОстаток на счете: {self.__balance} руб.")
            else:
                status = 'fail'
                self.operations_history.append([f"{status}, {now}, {type}, {amount}, {self.__balance}"])
                print(f"Недостаточно средств на счете. Попробуйте указать меньшую сумму\nДоступный баланс {self.__balance} руб.")

        return self.__balance

    def get_balance(self): #обращаемся к приватному атрибуту для получения текущего баланса
        return self.__balance

    def get_history(self): #получение истории операций
        for entry in self.operations_history:
            print(entry)

    def plot_history(self): #метод, который забирает историю операций и превращает ее в читабельный датафрейм Pandas
        self.data = pd.DataFrame(self.operations_history, columns=['1'])
        self.data = self.data['1'].str.split(',', expand=True)
        self.data.columns = ['Статус операции', 'Дата и время операции', 'Тип операции', 'Сумма', 'Конечный баланс']

        #типы данных меняем сразу, чтобы метод выдавал готовый датафрейм
        self.data['Дата и время операции'] = pd.to_datetime(self.data['Дата и время операции'])
        self.data['Сумма'] = self.data['Сумма'].astype(float)
        self.data['Конечный баланс'] = self.data['Конечный баланс'].astype(float)
        return self.data

    def analys(self): #метод для анализа истории транзакций по размеру и дате (выведет 5 операций. Только успешные!)
        temp = self.plot_history()
        for_analys=temp[temp['Статус операции'] == 'success'].sort_values(by=['Сумма', 'Дата и время операции'], ascending=False).reset_index(drop=True).head(5)
        return for_analys

Добавим несколько экземпляров класса - заведем несколько банковских счетов

In [ ]:
John: Account = Account(account_holder='John Black')
Jack: Account = Account(account_holder='Jack White')
Jenny: Account = Account(account_holder='Jenny Red', balance=500)

In [ ]:
Jenny.account_number() #посмотрим номер счета для одного пользователя

ACC-1003


In [ ]:
#проведем несколько операций внесения

John.deposit(200)
John.deposit(300)
John.deposit(500)

Внесено 200.0 руб.
Остаток на счете: 200.0 руб.
Внесено 300.0 руб.
Остаток на счете: 500.0 руб.
Внесено 500.0 руб.
Остаток на счете: 1000.0 руб.


1000.0

In [ ]:
#и проведем несколько операций снятия, одна из которых будет неудачной (недостаточно средств)
John.withdraw(100)
John.withdraw(2000)

Снято 100.0 руб.
Остаток на счете: 900.0 руб.
Недостаточно средств на счете. Попробуйте указать меньшую сумму
Доступный баланс 900.0 руб.


900.0

In [ ]:
John.get_balance() #выведем баланс для этого пользователя

900.0

In [ ]:
John.get_history() #и историю операций

['success, 2025-11-08 17:10:10.374561, deposit, 200.0, 200.0']
['success, 2025-11-08 17:10:10.374738, deposit, 300.0, 500.0']
['success, 2025-11-08 17:10:10.374803, deposit, 500.0, 1000.0']
['success, 2025-11-08 17:10:10.383513, withdraw, 100.0, 900.0']
['fail, 2025-11-08 17:10:10.384819, withdraw, 2000.0, 900.0']


Построим график, отражающий историю операций данного пользователя

Используем plotly, так как эта библиотека позволяет создать наглядные визуализации, которые могут быть адаптированы под ситуацию

Например, в зависимости от запросов по временной детализации можно использовать режим Zoom, менять масштаб оси

In [ ]:
new_data = John.plot_history()
new_data = new_data[new_data['Статус операции'] == 'success']
px.line(new_data, x='Дата и время операции', y='Конечный баланс', title = 'Изменение баланса банковского счета').update_layout(xaxis_title="Дата и время операции", yaxis_title="Баланс")

# Задание 2

Наследуем два класса

В каждом пропишем тип счета . Делаем это до __init___, т.к. это неизменяемый параметр класса

In [ ]:
class CheckingAccount(Account):
    account_type = 'Checking'
    def __init__(self, account_holder, balance = 0):
        super().__init__(account_holder, balance)

In [ ]:
class SavingAccount(Account):
    account_type = 'Saving'

    def __init__(self, account_holder, balance = 0):
        super().__init__(account_holder, balance)

    def withdraw(self, amount): #наследуем метод родительского класса, чтобы расширить его доп. условием
        if (0.5*self.get_balance()) < amount:
            amount = 0
            print('Нельзя снять больше половины имеющихся средств')
        super().withdraw(amount) #super() ставим в конце, чтобы новое условие отрабатывало первым (иначе средства все равно будут сниматься со счета)

    def apply_interest(self, rate): #новый метод, которого нет в родительском классе
        cashback = self.get_balance() * rate / 100
        return cashback
